### data setup

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import differential_evolution as de

In [25]:
raw_data = pd.read_csv('dados\MC carpel H 5-min.csv')
raw_data.head(5)

,time,temperature,tga,dtga,dtga_5
0,0,450.35,10.32687,-0.000092,-0.00046
1,5,450.85,10.32643,-0.000088,-0.00044
2,10,451.25,10.32595,-0.000096,-0.00048
3,15,451.65,10.32550,-0.000090,-0.00045
4,20,452.15,10.32502,-0.000096,-0.00048


In [26]:
raw_data.dtypes

time             int64
temperature    float64
tga            float64
dtga           float64
dtga_5         float64
dtype: object

### functions

In [27]:
def time_interval_decorator(function):
    import time
    def wrapper(*args):
        initial_time = time.time()
        function(*args)
        final_time = time.time()
        elapsed_time = final_time - initial_time
        print(f'Time elapsed: {elapsed_time} seconds.')
    return wrapper

def theorical_temperature(data):
    initial_temperature = data.temperature.iloc[0]
    heating_rate = 5 # K/min
    return initial_temperature + heating_rate/60 * data.time

def conversion(data):
    initial_mass = data.tga.iloc[0]
    final_mass = data.tga.iloc[-1]
    return (initial_mass - data.tga) / (initial_mass - final_mass)

def conversion_rate(order, data):
    return (1 - data.conversion)**order

def arrhenius(X, data):
    gas_constant = 8.314 # J/mol*K
    
    pre_exp_factor = X[0]
    activation_energy = X[1]
    order = X[2]

    rate = pre_exp_factor * np.exp( -activation_energy / (gas_constant * data.theoretical_temperature)) * conversion_rate(order, data)
    return rate

def rpi_model_for_mass(X, data):
    # component parameters = [(component_fraction), (pre_exp_factor), (activation_energy), (order)]
    hemicellulosis_params = X[:4]
    cellulosis_params = X[4:8]
    lignin_params = X[8:12]

    model_hemicellulosis = -hemicellulosis_params[0] * arrhenius(hemicellulosis_params[1:], data)
    model_cellulosis = -cellulosis_params[0] * arrhenius(cellulosis_params[1:], data)
    model_lignin = -lignin_params[0] * arrhenius(lignin_params[1:], data)

    complete_model = model_hemicellulosis + model_cellulosis + model_lignin
    return complete_model

def objective_function(X, data):
    data['err'] = (rpi_model_for_mass(X, data) - data['tga'])**2
    return data.err.sum()

### Guessing initial parameter intervals and hyper parameters for DE

In [28]:
# Bounds follows: [(component_fraction), (pre_exp_factor), (activation_energy), (order)]
hemicellulosis = [(0, 0.4), (0, 1e10), (0, 1e10), (0, 2.0)]
cellulosis = [(0, 0.5), (0, 1e10), (0, 1e10), (0, 2.0)]
lignin = [(0, 0.3), (0, 1e10), (0, 1e10), (0, 4.0)]
bounds = hemicellulosis + cellulosis + lignin

# DE hyper-parameters
strategy = 'best1bin'
maxiter = 1000
popsize = 15
tol = 1e-2
mutation = (0.5, 1)
recombination = 0.7

### Main script

In [29]:
# duplicating the data for safeguarding
data = raw_data.drop('dtga_5', axis=1).copy()

# conversion of mass over time
data['conversion'] = conversion(data)

# calculating theoretical temperature
data['theoretical_temperature'] = theorical_temperature(data)

data.head(5)

,time,temperature,tga,dtga,conversion,theoretical_temperature
0,0,450.35,10.32687,-0.000092,0.000000,450.350000
1,5,450.85,10.32643,-0.000088,0.000066,450.766667
2,10,451.25,10.32595,-0.000096,0.000137,451.183333
3,15,451.65,10.32550,-0.000090,0.000205,451.600000
4,20,452.15,10.32502,-0.000096,0.000276,452.016667


In [33]:
@time_interval_decorator
def main():
    results = de(objective_function, 
                bounds = bounds,
                args = (data, ),
                strategy = strategy,
                maxiter = maxiter,
                popsize = popsize,
                tol = tol,
                mutation = mutation,
                recombination = recombination,
                )
    params = results.x
    opt_fun = results.fun
    print(f'Params: {params}\nResult: {opt_fun}')

if __name__ == "__main__":
    main()

RuntimeError: func(x, *args) must return a scalar value